In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm.auto import tqdm
from copy import deepcopy
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

%load_ext autoreload
%autoreload 2

In [ ]:
try:
    import bib_lookup
except ModuleNotFoundError:
    sys.path.insert(0, "/home/wenhao/Jupyter/wenhao/workspace/bib_lookup/")
try:
    from torch_ecg.utils.misc import MovingAverage, list_sum
except ModuleNotFoundError:
    sys.path.insert(0, "/home/wenhao/Jupyter/wenhao/workspace/torch_ecg/")
    from torch_ecg.utils.misc import MovingAverage, list_sum

In [ ]:
from cfg import TrainCfg, ModelCfg
from trainer import CPSC2019Trainer, _MODEL_MAP
from model import ECG_SEQ_LAB_NET_CPSC2019, ECG_UNET_CPSC2019, ECG_SUBTRACT_UNET_CPSC2019
from dataset import CPSC2019

In [ ]:
TrainCfg.db_dir = "/home/wenhao/Jupyter/wenhao/data/CPSC2019/"

In [ ]:
ds_train = CPSC2019(TrainCfg, training=True, lazy=False)
ds_val = CPSC2019(TrainCfg, training=False, lazy=False)

## train CNN

In [ ]:
train_config = deepcopy(TrainCfg)
train_config.model_name = "seq_lab_cnn"

model_config = deepcopy(ModelCfg[train_config.model_name])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = ECG_SEQ_LAB_NET_CPSC2019(
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
# if torch.cuda.device_count() > 1:
#     model = DP(model)

model.to(device=device)

In [ ]:
trainer = CPSC2019Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
bmd = trainer.train()

In [ ]:
del bmd, trainer, model

## train CRNN

In [ ]:
train_config = deepcopy(TrainCfg)
train_config.model_name = "seq_lab_crnn"

model_config = deepcopy(ModelCfg[train_config.model_name])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_config

In [ ]:
model = ECG_SEQ_LAB_NET_CPSC2019(
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
# if torch.cuda.device_count() > 1:
#     model = DP(model)

model.to(device=device)

In [ ]:
trainer = CPSC2019Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
bmd = trainer.train()

In [ ]:
del bmd, trainer, model

## Train U-Net

In [ ]:
train_config = deepcopy(TrainCfg)
train_config.model_name = "unet"

model_config = deepcopy(ModelCfg[train_config.model_name])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = _MODEL_MAP[train_config.model_name](
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
# if torch.cuda.device_count() > 1:
#     model = DP(model)

model.to(device=device)

In [ ]:
trainer = CPSC2019Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
bmd = trainer.train()

In [ ]:
del bmd, trainer, model

# clear GPU

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

# gather results

In [ ]:
import seaborn as sns
from matplotlib.pyplot import cm
import matplotlib.patches as patches

sns.set()

plt.rcParams['xtick.labelsize']=28
plt.rcParams['ytick.labelsize']=28
plt.rcParams['axes.labelsize']=40
plt.rcParams['legend.fontsize']=24

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

markers = ["p", "v", "s", "d", "x", "*", "+", "$\heartsuit$"]
marker_size = 12

In [ ]:
from torch_ecg.utils.misc import MovingAverage, list_sum

In [ ]:
# ma = MovingAverage()

ma = lambda x: x

In [ ]:
df_cnn = pd.read_csv("./results/TorchECG_12-22_16-52_ECG_SEQ_LAB_NET_CPSC2019_adamw_amsgrad_LR_0.001_BS_32_multi_scopic.csv")
df_crnn = pd.read_csv("./results/TorchECG_12-22_17-13_ECG_SEQ_LAB_NET_CPSC2019_adamw_amsgrad_LR_0.001_BS_32_multi_scopic.csv")
df_unet = pd.read_csv("./results/TorchECG_12-22_17-48_ECG_UNET_CPSC2019_adamw_amsgrad_LR_0.001_BS_32_none.csv")

In [ ]:
df_cnn_train = df_cnn[df_cnn.part=="train"].dropna(subset=["qrs_score"])
df_crnn_train = df_crnn[df_crnn.part=="train"].dropna(subset=["qrs_score"])
df_unet_train = df_unet[df_unet.part=="train"].dropna(subset=["qrs_score"])
df_cnn_val = df_cnn[df_cnn.part=="val"].dropna(subset=["qrs_score"])
df_crnn_val = df_crnn[df_crnn.part=="val"].dropna(subset=["qrs_score"])
df_unet_val = df_unet[df_unet.part=="val"].dropna(subset=["qrs_score"])

In [ ]:
df_lr = df_unet[df_unet.part=="train"][["step", "lr"]].dropna(subset=["lr"])
df_lr["step"] = df_lr["step"]/(1600/32)

In [ ]:
fig, ax = plt.subplots(figsize=(24,16))

line_width = 4

spacing = 2

lines = []

lines.append(ax.plot(
    df_crnn_train.epoch.values[::spacing], ma(df_crnn_train.qrs_score.values)[::spacing],
    marker=markers[0], markersize=marker_size, linewidth=line_width, color=colors[0], label="crnn-train",
))
lines.append(ax.plot(
    df_cnn_train.epoch.values[::spacing], ma(df_cnn_train.qrs_score.values)[::spacing],
    marker=markers[1], markersize=marker_size, linewidth=line_width, color=colors[1], label="cnn-train",
))


lines.append(ax.plot(
    df_unet_train.epoch.values[::spacing], ma(df_unet_train.qrs_score.values)[::spacing],
    marker=markers[2], markersize=marker_size, linewidth=line_width, color=colors[3], label="unet-train",
))
lines.append(ax.plot(
    df_crnn_train.epoch.values[::spacing], ma(df_crnn_val.qrs_score.values)[::spacing],
    marker=markers[0], markersize=marker_size, linewidth=line_width, color=colors[0], ls="--", label="crnn-val",
))
lines.append(ax.plot(
    df_cnn_train.epoch.values[::spacing], ma(df_cnn_val.qrs_score.values)[::spacing],
    marker=markers[1], markersize=marker_size, linewidth=line_width, color=colors[1], ls="--", label="cnn-val",
))
lines.append(ax.plot(
    df_unet_train.epoch.values[::spacing], ma(df_unet_val.qrs_score.values)[::spacing],
    marker=markers[2], markersize=marker_size, linewidth=line_width, color=colors[3], ls="--", label="unet-val",
))
ax.set_ylim(0.55,1.05)
ax.set_xlabel("Epochs (n.u.)", fontsize=36)
ax.set_ylabel("QRS score (n.u.)", fontsize=36)

ax2 = ax.twinx()
lines.append(ax2.plot(
    df_lr.step.values, ma(df_lr.lr.values),
    color=colors[2], ls=":", linewidth=8, label="learning rate",
))
ax2.set_ylabel(r"$10^3\times$"+"Learning Rate (n.u.)")
ax2.set_ylim(-0.00025,0.00225)
ax2.set_yticks(np.arange(0,0.0025,0.0005).tolist())
ax2.set_yticklabels([f"{n:.1f}" for n in np.arange(0,0.0025,0.0005)*1000])

# ax.text(40,1.01, "max lr = 0.02", fontsize=30)
# ax.text(5,0.61, f"start lr = {df_lr.lr.values[0]:.5f}", fontsize=30)

lns = list_sum(lines[:-1])
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc="lower right", ncol=2, fontsize=30, bbox_to_anchor=(0.95, 0.1))

ax2.legend(loc="upper left", fontsize=30)
ax2.set_xlim(-5,105);

plt.savefig("./results/cpsc2019_nn_compare.svg", dpi=1200, bbox_inches="tight", transparent=False);
plt.savefig("./results/cpsc2019_nn_compare.pdf", dpi=1200, bbox_inches="tight", transparent=False);